# Model 1b - Horizontal stratigraphic gridded

This is the most simpel model of horizontally stacked layers and we will use a bunch of subgrids. We start by importing the necessary dependencies:

In [ ]:
# These two lines are necessary only if GemPy is not installed
import sys, os
#sys.path.append("../..")
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu"

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

Creating the model by importing the input data and displaying it:

In [ ]:
geo_data = gp.create_data([0,1000,0,1000,0,1000],resolution=[50,50,50], 
                        path_o = os.pardir+"/data/input_data/jan_models/model1_orientations.csv",
                        path_i = os.pardir+"/data/input_data/jan_models/model1_surface_points.csv") 

Setting and ordering the units and series:

In [ ]:
gp.map_series_to_surfaces(geo_data, {"Strat_Series": ('rock2','rock1'),"Basement_Series":('basement')})

In [ ]:
%matplotlib inline
gp.plot.plot_data(geo_data, direction='y')

Calculating the model:

In [ ]:
interp_data = gp.set_interpolation_data(geo_data, compile_theano=True,
                                        theano_optimizer='fast_run')

First create a combined grid of the total size we want to compute, and create the solution object

In [ ]:
combined_grid = gp.core.data.Grid()
combined_regular_grid = combined_grid.create_regular_grid(extent=[0, 1000, 0, 1000, 0, 1000], resolution=[100,100,100])
combined_solution = gp.core.model.Solution(grid=combined_grid, surfaces=geo_data.surfaces, series=geo_data.series)

Next, create the sub-grid objects, here we are asking for 2 divisons on the X, Y, Z axes.
This should result in 8 sub-grids extents and resolutions.

In [ ]:
grids = gp.grid_types.define_subgrid_parameters(combined_grid, 2, 2, 2)

In [ ]:
# should be 8
print(len(grids))
print(grids[0])

We now have a list of the extents and resolutions of the sub grids.
Now compute the model, ie do the 'fit'. 

In [ ]:
gp.compute_model(geo_data, set_solutions=False, compute_mesh=False) 

now that we have computed the model we can now 'evaluate' the model at our sub-grids

In [ ]:
sub_grids_and_solutions = []
for extent, res in grids:
    grid = gp.core.data.Grid()
    grid.create_regular_grid(extent=extent, resolution=res)
    geo_data.set_regular_grid(extent, res)
    sol = gp.compute_model(geo_data, output='geology', compute_mesh=False, set_solutions=False, 
                           reset_scalar=False, reset_block=False, reset_weights=False)
    sub_grids_and_solutions.append((grid,sol))

In [ ]:
x = gp.set_sub_grid_solutions_to_regular_grid(geo_data, combined_solution, combined_grid, sub_grids_and_solutions);

In [ ]:
geo_data.solutions.lith_block.shape

Displaying the result in x and y direction:

In [ ]:
%matplotlib inline
gp.plot.plot_section(geo_data, cell_number=10,
                         direction='x', show_all_data=True)

In [ ]:
%matplotlib inline
gp.plot.plot_section(geo_data, cell_number=5,
                    direction='y', show_data=True)
